# Extract Options Data
This notebook demonstrates how to extract options data from QuantConnect.

In [ ]:
from QuantConnect import *
from QuantConnect.Data.Market import *
from datetime import datetime, timedelta
import pandas as pd

class OptionsDataExtractor(QCAlgorithm):
    def Initialize(self):
        # Set start/end dates
        self.SetStartDate(2024, 1, 3)  # Wednesday
        self.SetEndDate(2024, 1, 3)
        self.SetCash(100000)
        
        # Add SPY equity and options
        equity = self.AddEquity("SPY")
        self.spy_options = self.AddOption("SPY")
        
        # Set our filter for options
        self.spy_options.SetFilter(self.FilterOptions)
        
        # Store our last print time to avoid duplicates
        self.last_print_time = None
    
    def FilterOptions(self, universe):
        # Only return options expiring today (0 DTE)
        return universe.IncludeWeeklys() \
                      .Strikes(-10, +10) \
                      .Expiration(0, 0)
    
    def OnData(self, slice):
        if not slice.OptionChains: 
            return
            
        # Get the chain
        chain = slice.OptionChains.Values[0]
        current_time = self.Time
        
        # Only print once per unique timestamp
        if self.last_print_time == current_time:
            return
            
        # Clear previous output
        from IPython.display import clear_output
        clear_output(wait=True)
        
        # Print header
        print(f"\nTime: {current_time}")
        print("Strike   Type   Bid    Ask   Last   IV ")
        print("-" * 45)
        
        # Sort contracts by strike
        sorted_contracts = sorted(chain, key=lambda x: (x.Strike, x.Right))
        
        # Print contract details
        for contract in sorted_contracts:
            print(f"{contract.Strike:7.1f} {contract.Right:6} "
                  f"{contract.BidPrice:6.2f} {contract.AskPrice:6.2f} "
                  f"{contract.LastPrice:6.2f} {contract.ImpliedVolatility:6.2%}")
            
        self.last_print_time = current_time

# Create and run the algorithm
algo = OptionsDataExtractor()
algo = QuantConnect.Python.Initialize()
algo.SetAlgorithm(algo)
results = algo.Run()